In [2]:
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model

emotions_korean = ["분노", "슬픔", "불안", "상처", "당황", "기쁨", "중립"]
emotions = ["angry", "sad", "anxious", "hurt", "embarrassed", "happy", "neutral"]


# model_path = "saved_models/v2_with_new_data_best_model.hdf5"
model_path = "모델/saved_models/efficientnetb1.hdf5"
model = load_model(model_path)
input_size = (224,224)
is_grey = False

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, ic = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # 얼굴 부분 자르기
            face = frame[y:y+h, x:x+w]
            try:
                face = cv2.resize(face, input_size) 
                if is_grey:
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) 
                face = face / 255.0 # 정규화
                face = np.expand_dims(face, axis=0)
                face = np.expand_dims(face, axis=-1)

                # 감정 예측
                prediction = model.predict(face)
                emotion = emotions[np.argmax(prediction)]

                # 텍스트로 감정 표시
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

            except Exception as e:
                print(e)
                continue

            # 얼굴에 박스 그리기
            mp_drawing.draw_detection(frame, detection)

    # 결과 보여주기
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 26ms/step
